In [45]:
import pandas as pd
import cv2 as cv
from ultralytics import YOLO
import torch
from pathlib import Path
import os
import shutil

# References

# pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu130
# https://docs.ultralytics.com/usage/python/
# https://docs.ultralytics.com/tasks/detect/#how-do-i-train-a-yolo11-model-on-my-custom-dataset
# https://docs.ultralytics.com/tasks/obb/#visual-samples
# https://docs.ultralytics.com/modes/train/#train-settings
# https://docs.ultralytics.com/tasks/obb/
# https://dipankarmedh1.medium.com/real-time-object-detection-with-yolo-and-webcam-enhancing-your-computer-vision-skills-861b97c78993

# cat dataset 
# https://www.kaggle.com/datasets/samuelayman/cat-dataset?resource=download


In [46]:
# first time using GPU acceleration. 
# GPU: NVIDIA GeForce RTX 3060 TI
if torch.cuda.is_available():
    print(torch.cuda.is_available())
    print(torch.version.cuda)
    print(torch.cuda.get_device_name(0))
else:
    print("Using CPU")    

True
13.0
NVIDIA GeForce RTX 3060 Ti


In [47]:
# pandas data preperation

current_dir = Path.cwd()
image_path = current_dir / "data" / "cat" / "images"
label_path = current_dir / "data" / "cat" / "labels"

df = pd.DataFrame(columns=["image", "bb_label"])

if image_path.is_dir():
    print("image dir found")
    if label_path.is_dir():
        print("label dir found")
        for img_file in image_path.iterdir():
            label_name = img_file.name
            label_name = label_name.replace("jpg","txt")
            # go to labels folder and read values from text file
            text_data = label_path / label_name
#########
# format to single class YOLO annotation format
            with open(text_data, "r") as file:
                lines = file.readline()
                updated = lines.replace("25", "0", 1)
            with open(text_data, "w") as file:
                file.write(updated)    
#########
            data_row = [img_file.resolve(), text_data.resolve()]
            df.loc[len(df)] = data_row

# print(f"Dataframe head \n {df.head}")
# print(f"Dataframe size \n {df.size}")

# randomly sample 80% of the pairs to train data
train_data = df.sample(frac=0.7, random_state= 1)
# 30%
test_data = df.drop(train_data.index)
# 15%
validation_data = test_data.sample(frac=0.5, random_state=1)
# now 15%
test_data = test_data.drop(validation_data.index)

print(f"train data count: {len(train_data)}")
print(f"test data count: {len(test_data)}")
print(f"validation data count: {len(validation_data)}")


image dir found
label dir found
train data count: 700
test data count: 150
validation data count: 150


In [48]:
# organise data for .yaml
datasets = current_dir / "dataset"

# if formatted data does not exist
if not datasets.is_dir():
    os.mkdir(datasets)
    images_folder = datasets / "images"
    os.mkdir(images_folder)
    labels_folder = datasets / "labels"
    os.mkdir(labels_folder)
    labels_folder_train = labels_folder / "train"
    labels_folder_test = labels_folder / "test"
    labels_folder_val = labels_folder / "val"
    os.mkdir(labels_folder_train)
    os.mkdir(labels_folder_test)
    os.mkdir(labels_folder_val)   
    images_folder_train = images_folder / "train"
    images_folder_test = images_folder / "test"
    images_folder_val = images_folder / "val"
    os.mkdir(images_folder_train)
    os.mkdir(images_folder_test)
    os.mkdir(images_folder_val)

        # populate testing data
    for row in train_data.itertuples(index=False):
        try:
            shutil.copy(row[0], images_folder_train)
            shutil.copy(row[1], labels_folder_train)
        
        except shutil.SameFileError:
            print("Source and destination represents the same file.")

        except PermissionError:
            print("Permission denied.")

        except:
            print("Error in process")

    for row in test_data.itertuples(index=False):
        try:
            shutil.copy(row[0], images_folder_test)
            shutil.copy(row[1], labels_folder_test)
        
        except shutil.SameFileError:
            print("Source and destination represents the same file.")

        except PermissionError:
            print("Permission denied.")

        except:
            print("Error in process")

    for row in validation_data.itertuples(index=False):
        try:
            shutil.copy(row[0], images_folder_val)
            shutil.copy(row[1], labels_folder_val)
        
        except shutil.SameFileError:
            print("Source and destination represents the same file.")

        except PermissionError:
            print("Permission denied.")

        except:
            print("Error in process")

In [50]:
#load in pre trained model
model = YOLO("yolo11n.pt")
model.to('cuda')
# train model with cat dataset
# https://docs.ultralytics.com/modes/train/#train-settings
# testing for overfitting
model.train(data="my-dataset.yaml", 
            epochs=300, 
            optimizer="AdamW",
            patience=50
            )

New https://pypi.org/project/ultralytics/8.3.228 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=my-dataset.yaml, epochs=300, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

train: Scanning C:\Users\rrpar\OneDrive\Cat_Detection\dataset\labels\train.cache... 700 images, 0 backgrounds, 0 corrupt: 100%|██████████| 700/700 [00:00<?, ?it/s]
val: Scanning C:\Users\rrpar\OneDrive\Cat_Detection\dataset\labels\val.cache... 150 images, 0 backgrounds, 0 corrupt: 100%|██████████| 150/150 [00:00<?, ?it/s]


Plotting labels to runs\detect\train5\labels.jpg... 
optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train5
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/300      2.25G      1.645      2.418      1.895         28        640: 100%|██████████| 44/44 [00:05<00:00,  7.81it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  8.72it/s]

                   all        150        150     0.0272       0.12     0.0153    0.00357


...

                   all        150        150      0.817      0.835      0.881      0.631
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 238, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



288 epochs completed in 0.441 hours.
Optimizer stripped from runs\detect\train5\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train5\weights\best.pt, 5.5MB

Validating runs\detect\train5\weights\best.pt...
Ultralytics 8.3.107  Python-3.12.8 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  7.75it/s]


                   all        150        150      0.801      0.884      0.889      0.641
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002A49D2C1D30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [ ]:
results = model.val(data="my-dataset.yaml")

print("Class indices with average precision:", results.ap_class_index)
print("Average precision for all classes:", results.box.all_ap)
print("Average precision:", results.box.ap)
print("Average precision at IoU=0.50:", results.box.ap50)
print("Class indices for average precision:", results.box.ap_class_index)
print("Class-specific results:", results.box.class_result)
print("F1 score:", results.box.f1)
print("F1 score curve:", results.box.f1_curve)
print("Overall fitness score:", results.box.fitness)
print("Mean average precision:", results.box.map)
print("Mean average precision at IoU=0.50:", results.box.map50)
print("Mean average precision at IoU=0.75:", results.box.map75)
print("Mean average precision for different IoU thresholds:", results.box.maps)
print("Mean results for different metrics:", results.box.mean_results)
print("Mean precision:", results.box.mp)
print("Mean recall:", results.box.mr)
print("Precision:", results.box.p)
print("Precision curve:", results.box.p_curve)
print("Precision values:", results.box.prec_values)
print("Specific precision metrics:", results.box.px)
print("Recall:", results.box.r)
print("Recall curve:", results.box.r_curve)

Ultralytics 8.3.107  Python-3.12.8 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\rrpar\OneDrive\Cat_Detection\dataset\labels\val.cache... 150 images, 0 backgrounds, 0 corrupt: 100%|██████████| 150/150 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  2.93it/s]


                   all        150        150       0.88      0.881      0.934      0.743
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\train42
Class indices with average precision: [0]
Average precision for all classes: [[    0.93365     0.92553     0.90766     0.89318     0.85351     0.81614     0.73922      0.6385     0.49976     0.21972]]
Average precision: [    0.74269]
Average precision at IoU=0.50: [    0.93365]
Class indices for average precision: [0]
Class-specific results: <bound method Metric.class_result of ultralytics.utils.metrics.Metric object with attributes:

all_ap: array([[    0.93365,     0.92553,     0.90766,     0.89318,     0.85351,     0.81614,     0.73922,      0.6385,     0.49976,     0.21972]])
ap: array([    0.74269])
ap50: array([    0.93365])
ap_class_index: array([0])
curves: []
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,

In [ ]:
#proof of concept with footage recorded on drone camera

#PICTA0001 has trouble finding cat in room
#PICTA0004 for low to very high noise
#PICTA0005 for high noise
#PICTA0006 for low noise --> note as the camera moves further away from the cat it has less confidence
#PICTA0007 for low noise
#PICTA0008 further distance with osd and gradual noise. decent confidence but tracks other objects as cat such as cealing fan
#PICTA0012 for on screen display and rotation
#PICTA0016 --> note footage views carpet texture as cat when close

tracking_model = YOLO("runs/detect/train3/weights/best.pt")
class_names = ["cat"]
drone_footage = current_dir / "drone_footage" / "PICT0016.AVI"
capture_cam = cv.VideoCapture(drone_footage)

while capture_cam.isOpened():
    success, frame = capture_cam.read()

    if success:
        results = tracking_model.track(frame, persist=True)
        annotate_frame = results[0].plot()
        cv.imshow("Cat tracking", annotate_frame)
    # pass frames to yolo for object detection and treat as a continuous input stream
    # https://docs.ultralytics.com/modes/track/#tracker-arguments
        if cv.waitKey(1) == ord("q"):
            break
    else:
        break    
capture_cam.release()
cv.destroyAllWindows()




0: 512x640 (no detections), 31.1ms
Speed: 2.5ms preprocess, 31.1ms inference, 0.4ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 41.9ms
Speed: 2.5ms preprocess, 41.9ms inference, 2.9ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 77.8ms
Speed: 3.6ms preprocess, 77.8ms inference, 6.2ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 42.0ms
Speed: 5.0ms preprocess, 42.0ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 6.9ms
Speed: 1.2ms preprocess, 6.9ms inference, 0.4ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 6.9ms
Speed: 1.2ms preprocess, 6.9ms inference, 0.4ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 6.8ms
Speed: 1.1ms preprocess, 6.8ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 6.8ms
Speed: 1.1ms preprocess, 6.8ms inference